Задача Кластеризации

Кластеризация (англ. cluster analysis) — задача группировки множества объектов на подмножества (кластеры) таким образом, чтобы объекты из одного кластера были более похожи друг на друга, чем на объекты из других кластеров по какому-либо критерию.

Задача кластеризации относится к классу задач обучения без учителя.

![](2021-07-26-19-41-21.png)

Пример кластеров

Цели Кластеризации

1. Классификация объектов. Попытка понять зависимости между объектами путем выявления их кластерной структуры. Разбиение выборки на группы схожих объектов упрощает дальнейшую обработку данных и принятие решений, позволяет применить к каждому кластеру свой метод анализа (стратегия «разделяй и властвуй»). В данном случае стремятся уменьшить число кластеров для выявления наиболее общих закономерностей;

2. Сжатие данных. Можно сократить размер исходной выборки, взяв один или несколько наиболее типичных представителей каждого кластера. Здесь важно наиболее точно очертить границы каждого кластера, их количество не является важным критерием;

3. Обнаружение новизны (обнаружение шума). Выделение объектов, которые не подходят по критериям ни в один кластер. Обнаруженные объекты в дальнейшем обрабатывают отдельно.

Методы Кластеризации

1. Графовые алгоритмы кластеризации. Наиболее примитивный класс алгоритмов. В настоящее время практически не применяется на практике;

2. Вероятностные алгоритмы кластеризации. Каждый объект из обучающей выборки относится к каждому из кластеров с определенной степенью вероятности:
EM-алгоритм;

3. Иерархические алгоритмы кластеризации. Упорядочивание данных путем создания иерархии вложенных кластеров;
Алгоритм k-средних (англ. k-means). Итеративный алгоритм, основанный на минимизации суммарного квадратичного отклонения точек кластеров от центров этих кластеров;

4. Распространение похожести (англ. affinity propagation). Распространяет сообщения о похожести между парами объектов для выбора типичных представителей каждого кластера;

5. Сдвиг среднего значения (англ. mean shift). Выбирает центроиды кластеров в областях с наибольшей плотностью;

6. Спектральная кластеризация (англ. spectral clustering). Использует собственные значения матрицы расстояний для понижения размерности перед использованием других методов кластеризации;

7. Основанная на плотности пространственная кластеризация для приложений с шумами (англ. Density-based spatial clustering of applications with noise, DBSCAN). Алгоритм группирует в один кластер точки в области с высокой плотностью. Одиноко расположенные точки помечает как шум.

Задача — разделить ирисы на группы в зависимости от признаков цветка.

Устанавливаем ML.NET

In [ ]:
#r "nuget:Microsoft.ML"


Installed Packages Microsoft.ML, 1.6.0 microsoft.ml.cpumath, 1.6.0 microsoft.ml.dataview, 1.6.0 newtonsoft.json, 10.0.3 system.codedom, 4.4.0

Начинаем использовать установленные библиотеки ML.NET

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;

Используем IO для загрузки данных

In [ ]:
using System.IO;

static readonly string _dataPath = Path.Combine(Environment.CurrentDirectory, "Data", "iris.data.txt");

Класс для хранения данных об ирисах

Атрибут LoadColumn используется для указания индексов исходных столбцов в файле набора данных.

In [ ]:
public class IrisData
{
    [LoadColumn(0)]
    public float SepalLength;

    [LoadColumn(1)]
    public float SepalWidth;

    [LoadColumn(2)]
    public float PetalLength;

    [LoadColumn(3)]
    public float PetalWidth;
}

Класс для отображения результата

Атрибут ColumnName используется для привязки полей PredictedClusterId и Distances к столбцам PredictedLabel и Score соответственно. 

In [ ]:
public class ClusterPrediction
{
    [ColumnName("PredictedLabel")]
    public uint PredictedClusterId;

    [ColumnName("Score")]
    public float[] Distances;
}

Класс Microsoft.ML.MLContext представляет среду машинного обучения и предоставляет механизмы для ведения журнала и точек входа для загрузки данных, обучения модели, прогнозирования и других задач.

Универсальный метод расширения MLContext.Data.LoadFromTextFile выводит схему набора данных на основе предоставленного типа IrisData и возвращает интерфейс IDataView, который можно использовать в качестве входных данных для преобразования.

In [ ]:
var mlContext = new MLContext(seed: 0);
IDataView dataView = mlContext.Data.LoadFromTextFile<IrisData>(_dataPath, hasHeader: false, separatorChar: ',');

Делим исходные данные на 3 кластера

1. Объединяем загруженные столбцы в один столбец "Функции", который используется тренером кластеризации;
2. Для обучения используем тренер KMeansTrainer с помощью алгоритма кластеризации k-means++.

k-means++ — улучшенная версия алгоритма кластеризации k-means. Суть улучшения заключается в нахождении более «хороших» начальных значений центроидов кластеров.

In [ ]:
string featuresColumnName = "Features";
var pipeline = mlContext.Transforms
    .Concatenate(featuresColumnName, "SepalLength", "SepalWidth", "PetalLength", "PetalWidth")
    .Append(mlContext.Clustering.Trainers.KMeans(featuresColumnName, numberOfClusters: 3));

Обучаем модель

In [ ]:
var model = pipeline.Fit(dataView);

Чтобы получить прогноз, используем класс PredictionEngine<TSrc,TDst>, который принимает экземпляры входного типа через конвейер преобразователя и создает экземпляры выходного типа.

In [ ]:
var predictor = mlContext.Model.CreatePredictionEngine<IrisData, ClusterPrediction>(model);

Создаём тестовые данные

In [ ]:
static class TestIrisData
{
    internal static readonly IrisData Setosa = new IrisData
    {
        SepalLength = 5.1f,
        SepalWidth = 3.5f,
        PetalLength = 1.4f,
        PetalWidth = 0.2f
    };
}

Проводим предположение и выводим его! 

In [ ]:
var prediction = predictor.Predict(TestIrisData.Setosa);
Console.WriteLine($"Cluster: {prediction.PredictedClusterId}");
Console.WriteLine($"Distances: {string.Join(" ", prediction.Distances)}");

Cluster: 2


Distances: 25,314243 0,021591187 11,645229
